In [1]:
import os
import pandas as pd
import numpy as np

train_data = pd.read_csv("data/training_data_utf8.csv")
val_data = pd.read_csv("data/validation_data_utf8.csv")

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
date = train_data.transaction_ym.apply(pd.to_datetime, format="%Y-%m")
relativeShare = train_data.plot_area * train_data.plot_share

In [3]:
train_data["relativeShare"] = relativeShare
train_data["transaction_month"] = date.dt.month
train_data["transaction_year"] = date.dt.year

In [4]:
train_data_clean = train_data[["price_per_m2", "transaction_year", "relativeShare", "property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]
train_data_clean = train_data_clean.dropna()
train_data_clean.shape

(44260, 9)

In [5]:
#from sklearn.model_selection import train_test_split
#train_set, test_set = train_test_split(train_data_clean, test_size=0.2, random_state=42)

In [6]:
#data = train_set.drop("price_per_m2", axis=1)
#labels = train_set["price_per_m2"].copy()

In [7]:
#test_data = test_set.drop("price_per_m2", axis=1)
#test_labels = test_set["price_per_m2"].copy()

In [8]:
data = train_data_clean.drop("price_per_m2", axis=1)
labels = train_data_clean["price_per_m2"].copy()

In [9]:
train_data_cat = train_data_clean[["property_type", "cadastral", "contract_type", "land_use", "protection_zone", "building_ban"]]

In [10]:
train_data_num = train_data_clean[["transaction_year", "relativeShare"]]

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

In [12]:
from future_encoders import ColumnTransformer
from future_encoders import OneHotEncoder

num_attribs = list(train_data_num)
cat_attribs = list(train_data_cat)

fullPipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs)
])

In [13]:
data_prepared = fullPipeline.fit_transform(data)
data_prepared

<44260x177 sparse matrix of type '<class 'numpy.float64'>'
	with 354080 stored elements in Compressed Sparse Row format>

In [14]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(data_prepared, labels)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [15]:
some_data = data.iloc[:5]
some_labels = labels.iloc[:5]
some_data_prepared = fullPipeline.transform(some_data)
print("Predictions:", lin_reg.predict(some_data_prepared))
print('Labels:', list(some_labels))

Predictions: [ 934.74582264  778.94789359  911.61095942 1326.79912205  729.22052011]
Labels: [224.351483991587, 357.99522673031, 636.474908200734, 906.095551894563, 167.941497856602]


In [16]:
from sklearn.metrics import mean_squared_error

predictions = lin_reg.predict(data_prepared)
lin_mse = mean_squared_error(labels, predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

1913.4007810532614

In [17]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor()
forest_reg.fit(data_prepared, labels)

/home/loherzberger/Projects/ML/venv/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [18]:
predictions = forest_reg.predict(data_prepared)
for_mse = mean_squared_error(labels, predictions)
for_rmse = np.sqrt(for_mse)
for_rmse

969.1395131829798

In [19]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(forest_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)

In [20]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard Deviation", scores.std())
          
display_scores(forest_rmse_scores)

Scores: [1859.75621004 1540.93139783 2401.95303047 3541.12293652 3194.71085165
 1776.9375259  1568.78320759 2238.30434378  779.11014249  639.73088179]
Mean: 1954.1340528069577
Standard Deviation 883.5572127509997


In [21]:
scores = cross_val_score(forest_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
linear_rmse_scores = np.sqrt(-scores)

display_scores(linear_rmse_scores)

Scores: [1911.17415989 1472.13277139 2463.94348636 3538.46259706 3244.22729352
 1911.32714461 1791.77066719 2234.1690892   772.43435707  850.81724314]
Mean: 2019.0458809414783
Standard Deviation 857.1297785723642


In [22]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(data_prepared, labels)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [23]:
predictions = tree_reg.predict(data_prepared)
tree_mse = mean_squared_error(labels, predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

78.52238929936152

In [24]:
scores = cross_val_score(tree_reg, data_prepared, labels, scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores)

display_scores(tree_rmse_scores)

Scores: [2159.01377167 1758.13819806 2761.15924266 4064.76756987 5900.21273687
 1930.73304866 2298.57661821 2336.46948604  867.83603981  721.72319941]
Mean: 2479.8629911261573
Standard Deviation 1447.6992072126484


In [25]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}
]

grid_search = GridSearchCV(forest_reg, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(data_prepared, labels)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [26]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

array([1.36272474e-01, 3.90498914e-01, 4.93669600e-03, 1.53651685e-03,
       3.88177736e-02, 5.54947627e-03, 9.71529342e-03, 5.70805554e-07,
       5.63513616e-04, 2.19579932e-04, 2.05872045e-03, 2.87530926e-02,
       3.25063427e-03, 2.37102723e-04, 1.22032618e-05, 3.58519027e-04,
       1.51213869e-04, 1.14314023e-02, 6.26376082e-06, 8.55626984e-03,
       5.49750658e-04, 1.19096760e-03, 8.12993880e-04, 2.49420146e-04,
       2.67712768e-04, 2.47783311e-04, 1.53017131e-03, 8.66855202e-04,
       9.98270099e-04, 2.54879699e-04, 1.52145848e-03, 1.60648804e-03,
       9.87106251e-04, 1.43911208e-03, 2.30775303e-04, 3.03011005e-04,
       2.86161325e-04, 5.97258421e-04, 1.01930093e-03, 9.23336723e-05,
       1.96961145e-04, 3.06516165e-04, 1.87399619e-03, 2.89804119e-03,
       5.80503370e-04, 1.09247929e-03, 2.73012167e-04, 1.29236706e-01,
       2.79653697e-03, 9.34821392e-04, 1.84796947e-04, 2.28105210e-06,
       3.85412655e-03, 2.17416525e-03, 8.09848941e-05, 1.01805184e-03,
      

In [27]:
attributes = num_attribs + cat_attribs
sorted(zip(feature_importances, attributes), reverse=True)

[(0.39049891398502784, 'relativeShare'),
 (0.13627247418377123, 'transaction_year'),
 (0.0388177735815147, 'contract_type'),
 (0.009715293415024603, 'protection_zone'),
 (0.005549476271041922, 'land_use'),
 (0.004936695996971335, 'property_type'),
 (0.0015365168454749144, 'cadastral'),
 (5.708055538599619e-07, 'building_ban')]

In [29]:
train_data_cleaned_further = train_data_clean[["price_per_m2", "relativeShare", "transaction_year", "contract_type", "protection_zone", "land_use", "property_type"]]

In [30]:
data_elix = train_data_cleaned_further.drop("price_per_m2", axis=1)
labels_elix = train_data_cleaned_further["price_per_m2"].copy()

In [34]:
train_data_cat_elix = train_data_cleaned_further[["property_type", "contract_type", "land_use", "protection_zone"]]
train_data_num_elix = train_data_cleaned_further[["transaction_year", "relativeShare"]]

num_attribs_elix = list(train_data_num_elix)
cat_attribs_elix = list(train_data_cat_elix)

fullPipeline_elix = ColumnTransformer([
    ("num", num_pipeline, num_attribs_elix),
    ("cat", OneHotEncoder(), cat_attribs_elix)
])

In [35]:
data_prepared_elix = fullPipeline_elix.fit_transform(data_elix)

In [36]:
forest_reg_elix = RandomForestRegressor()
forest_reg_elix.fit(data_prepared_elix, labels_elix)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [37]:
scores = cross_val_score(forest_reg_elix, data_prepared_elix, labels_elix, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores_elix = np.sqrt(-scores)

In [38]:
display_scores(forest_rmse_scores_elix)

Scores: [2741.18096957 2215.21314521 2807.61262223 3593.01281605 4346.25589448
 1852.67378864 1710.34418982 2274.12936439  788.35924504  729.34234488]
Mean: 2305.8124380298477
Standard Deviation 1077.900494571515
